In [4]:
from rag_module.rag import RAG
import os

os.environ["embedding_path"] = "./embeddings/ELOQ_news_partial_2/"

In [5]:
#rag = RAG(scope_model_id='dfa843b7e7b740f8b2e4f9098a7884fd')
#rag = RAG(scope_model_id='5217cda942f54213aeb8d18a8dcdaa79')
rag = RAG(scope_model_id='f87519525e9146b8b026c4ff27cec00c') # 2 labels


INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
Batches: 100%|██████████| 22/22 [00:10<00:00,  2.18it/s]


Loading default flashrank model for language en
Default Model: ms-marco-MiniLM-L-12-v2
Loading FlashRankRanker model ms-marco-MiniLM-L-12-v2 (this message can be suppressed by setting verbose=0)
Loading model FlashRank model ms-marco-MiniLM-L-12-v2...
Successfully loaded model from run f87519525e9146b8b026c4ff27cec00c
Loaded features: ['top_reranker_score', 'top_sparse_score', 'top_dense_score']


In [6]:
rag.embedder._load_index_from_chunkdata()

Batches: 100%|██████████| 22/22 [00:09<00:00,  2.28it/s]


In [3]:
rag.query("What are the options for hosting a graph database in Azure?")

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.64it/s]


Features for scope prediction: [np.float32(0.9945979), np.float64(22.199729732896213), np.float32(0.29875493)]


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


{'chunks': [{'filename': 'azure-managed-databases.md',
   'text': '## Graph database  \nFor graph databases there is a single managed offering in Azure.',
   'vector_id': 1416,
   'distance': np.float32(0.29875493),
   'score': np.float32(0.9945979)},
  {'filename': 'azure-managed-databases.md',
   'text': '### Azure Cosmos DB (Gremlin API)  \nWith the [Gremlin API in Azure Cosmos DB](https://docs.microsoft.com/en-us/azure/cosmos-db/graph-introduction) you can create and query large graphs.  \n> **Guidance:** Use **Azure Cosmos DB with the Gremlin API** for graph databases.\n> **Why:** This is the only managed graph database offered in Azure.',
   'vector_id': 1417,
   'distance': np.float32(0.57931983),
   'score': np.float32(0.9836745)},
  {'filename': 'azure-managed-databases.md',
   'text': '### Alternative graph database  \nIf _Cosmos DB_ is too pricy, it is an option to host a Neo4J environment with virtual machines or containers. For enterprise features, additional licenses migh